In [ ]:
"""
Set up the environment:

(1) pip install jupyterlab==2.2.5

(2) nodejs:
sudo apt update
curl -sL https://deb.nodesource.com/setup_10.x | sudo -E bash -
sudo apt install nodejs
node -v

(3) npm:
sudo apt-get install nodejs-dev node-gyp libssl1.0-dev
sudo apt-get install npm

If you are on MacOS: install both nodejs and npm from https://nodejs.org.en

(4) Install the extensions "toc" and "collapsible_headings" 
"""

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import itertools
import warnings
# warnings.filterwarnings('ignore') 

import seaborn as sns # If you got the error "No module named 'numpy.testing.decorators'", downgrade numpy to 1.17.0
from matplotlib import pyplot as plt

# If you encounter errors when installing xgboost:
# try to install libomp by "brew install libomp" on MacOS or install cmake by "pip3 install cmake"
# pip install xgboost==1.5.2
import xgboost as xgb

# pip install scikit-learn==0.24.2
from sklearn import tree
from sklearn.metrics import (
    mutual_info_score, confusion_matrix, 
    accuracy_score, recall_score, precision_score, 
    precision_recall_fscore_support, make_scorer, 
    plot_roc_curve, plot_precision_recall_curve, 
    classification_report, roc_auc_score, mean_squared_error
)
from sklearn.metrics.cluster import contingency_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.utils.class_weight import compute_class_weight
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.svm import SVC, SVR

plt.rcParams["figure.figsize"] = (20, 20)
sns.set(rc={'figure.figsize': (20, 20)})
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)
pd.options.mode.chained_assignment = None

def get_clf_metrics(y_true, y_pred, n_labels):
    result = pd.DataFrame(index=list(range(n_labels)), columns=['accuracy', 'precision', 'recall', 'f-score'])
    acc = accuracy_score(y_true, y_pred)
    array = precision_recall_fscore_support(y_true, y_pred, average=None)
    for i in range(n_labels):
        result.loc[i, 'accuracy'] = acc
        result.loc[i, 'precision'] = array[0][i]
        result.loc[i, 'recall'] = array[1][i]
        result.loc[i, 'f-score'] = array[2][i]
    return result

def get_confusion_matrix(y_true, y_pred, n_labels):
    result = pd.DataFrame(confusion_matrix(y_true, y_pred), index=list(range(n_labels)), columns=list(range(n_labels)))
    result.index.name = 'true'
    result.columns.name = 'pred'
    return result

# Load data

In [ ]:
# Good datasets you can try: https://towardsdatascience.com/all-the-datasets-you-need-to-practice-data-science-skills-and-make-a-great-portfolio-74f2eb53b38a
df = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/home_data.csv', sep=',', index_col=False)
df

In [ ]:
# Specify the data types for some columns
# It is a good practice to define several lists of columns

"""
zipcode is usually a very important variable, but it is a categorical variable with too many categories, 
so we usually need to do "binning", e.g., group them according to the first three digits, 
but here for simplicity, we just ignore it

yr_renovated has many zeros (zero inflated) and few large numbers (years), 
so we usually should make it a categorical variable with three categories, e.g., (1) "Never renovated", (2) "Renovated within 5 years", (3) "Renovated more than 5 years ago",
but here for simplicity, we just ignore it
"""
ignore_cols = ['id', 'zipcode', 'yr_renovated']

dt_cols = ['date']
for col in dt_cols:
    df[col] = pd.to_datetime(df[col])  # You can't directly do this in the above "dtype"
    
res_cols = ['price']
df['price'] = pd.Categorical((df['price'] > df['price'].mean()).astype(int), ordered=True) # We make it a binary classification problem

cat_cols = ['waterfront', 'view', 'condition', 'grade']
for col in cat_cols:
    df[col] = pd.Categorical(df[col], ordered=True)  # To keep the order of categorical variables, we set ordered = True
    
cont_cols = [col for col in df.columns if col not in ignore_cols + dt_cols + res_cols + cat_cols]

df

In [ ]:
df_original = df.copy()

# Manually create some missing values
np.random.seed(0)
df.loc[np.random.choice(df.index, 10), 'sqft_living'] = np.nan
df.loc[np.random.choice(df.index, 20), 'sqft_above'] = np.nan
df.loc[np.random.choice(df.index, 5), 'grade'] = np.nan
df.loc[np.random.choice(df.index, 30), 'waterfront'] = np.nan

# Exploratory analysis

## Basic summary 

In [ ]:
# Make sure the columns have the correct data types and check whether there are missing values
df.info()

In [ ]:
df.describe(include='all', datetime_is_numeric=True)

## Visualization

### Plots for each individual variable

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(40, 35))

fig.suptitle('Plots for each individual variable')
fig.tight_layout()
fig.subplots_adjust(top=0.95)

axes = axes.flatten()
for i, col in enumerate(res_cols + cat_cols + cont_cols):
    if df[col].dtype.name == 'category':
        sns.countplot(ax=axes[i], x=col, data=df)
    else:
        sns.histplot(ax=axes[i], x=col, data=df)

### Plots for each pair of variables

In [ ]:
"""
Good reference: 
https://seaborn.pydata.org/tutorial/categorical.html
https://data-science-master.github.io/lectures/09_python/09_matplotlib.html
https://seaborn.pydata.org/generated/seaborn.barplot.html

    Variable on Y                    Variable on X                   Plot
(1) continuous (int or float)        continuous (int or float)       scatter plot / regplot with 95% CI
(2) continuous (int or float)        category (ordered or not)       swarmplot / stripplot / violinplot / boxplot 
(3) category (ordered or not)        continuous (int or float)       swarmplot / stripplot / violinplot / boxplot / histplot with the categorical variable as hue
(4) category (ordered or not)        category (ordered or not)       confusion table / bar plot with hue (bar height is the count)

You may also consider to incorporate some numeric characteristics (e.g., mean, median):

For (1), you can also use lineplot, where x is one variable and y is some characteristic of the other variable

In seaborn, lineplot has an argument "estimator" for aggregating across multiple observations of the variable on Y at the same X level. 
If it is None, all observations will be drawn

For (2) (3) (4), you can also use bar plot, where each class of a categorical variable corresponds to a bar and 
the bar height / length is some characteristic of the other variable (you can specify the "estimator" argument in barplot)

In seaborn, the barplot() function operates on a full dataset and applies a function to obtain the estimate (taking the mean by default). 
When there are multiple observations in each category, it also uses bootstrapping to compute a confidence interval around the estimate, which is plotted using error bars
"""

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 7))

# Case (1) scatter plot
sns.scatterplot(ax=axes[0], x='sqft_above', y='sqft_living', data=df)

# Case (1) regplot
sns.regplot(ax=axes[1], x='sqft_above', y='sqft_living', data=df)

# Case (2) lineplot
sns.lineplot(ax=axes[2], x='sqft_above', y='sqft_living', data=df, estimator=np.mean)

In [ ]:
fig, axes = plt.subplots(1, 6, figsize=(60, 7))

# Case (2) swarmplot
sns.swarmplot(ax=axes[0], x='grade', y='sqft_above', data=df.iloc[:100, :])  # swarmplot can't show too many points, so we select only the first 100 points

# Case (2) stripplot
sns.stripplot(ax=axes[1], x='grade', y='sqft_above', data=df)

# Case (2) violinplot
sns.violinplot(ax=axes[2], x='grade', y='sqft_above', data=df)

# Case (2) boxplot
sns.boxplot(ax=axes[3], x='grade', y='sqft_above', data=df)

# Case (2) barplot with mean of sqft_above as bar height
sns.barplot(ax=axes[4], x='grade', y='sqft_above', data=df, estimator=np.mean)

# Case (2) barplot with median of sqft_above as bar height
sns.barplot(ax=axes[5], x='grade', y='sqft_above', data=df, estimator=np.median)

In [ ]:
fig, axes = plt.subplots(1, 7, figsize=(70, 7))

# Case (3) swarmplot
sns.swarmplot(ax=axes[0], x='sqft_above', y='grade', data=df.iloc[:100, :])  # swarmplot can't show too many points, so we select only the first 100 points

# Case (3) stripplot
sns.stripplot(ax=axes[1], x='sqft_above', y='grade', data=df)

# Case (3) violinplot
sns.violinplot(ax=axes[2], x='sqft_above', y='grade', data=df)

# Case (3) boxplot
sns.boxplot(ax=axes[3], x='sqft_above', y='grade', data=df)

# Case (3) histplot with the categorical variable as hue
sns.histplot(ax=axes[4], x='sqft_above', hue='grade', data=df)

# Case (3) barplot with mean of sqft_above as bar length
sns.barplot(ax=axes[5], x='sqft_above', y='grade', data=df, estimator=np.mean)

# Case (3) barplot with median of sqft_above as bar length
sns.barplot(ax=axes[6], x='sqft_above', y='grade', data=df, estimator=np.median)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 7))

# Case (4) confusion table
confusion_table = pd.crosstab(df['grade'], df['condition'], rownames=['grade'], colnames=['condition'])
# confusion_table = pd.crosstab(df['grade'], [df['condition'], df['view']], rownames=['grade'], colnames=['condition', 'view'], margins=True)
sns.heatmap(ax=axes[0], data=confusion_table, cmap='Reds', annot=True)

# Case (4) bar plot with hue (bar height is the count)
sns.countplot(ax=axes[1], x='grade', hue='condition', data=df)

# Case (4) barplot with mean of waterfront (rate of waterfront) as bar height
df_copy = df.copy()
df_copy['waterfront'] = df_copy['waterfront'].astype('float')  # We must make waterfront to be numeric, otherwise we can't calculate its mean
sns.barplot(ax=axes[2], x='grade', y='waterfront', data=df_copy, estimator=np.mean)

### Plots for each individual variable and each pair of variables simultaneously

In [ ]:
"""
Good reference: 
https://seaborn.pydata.org/generated/seaborn.PairGrid.html
https://seaborn.pydata.org/generated/seaborn.pairplot.html

We always recommend to separate into (1) cont vs cont (2) cat vs cont (3) cont vs cat (4) cat vs cat
as we need different kinds of plots for each of these four cases

In general you have the following choices:

Response    How to regard response in plot         Types of other variables
---------------------------------------------------------------------------
cat         hue                                    cont vs cont
cat         hue                                    cat vs cont
cat         hue                                    cont vs cat
cat         hue                                    cat vs cat
---------------------------------------------------------------------------
cat         cat (plotted only when there is cat)   cont vs cont
cat         cat (plotted only when there is cat)   cat vs cont
cat         cat (plotted only when there is cat)   cont vs cat
cat         cat (plotted only when there is cat)   cat vs cat
---------------------------------------------------------------------------
cont        hue                                    cont vs cont
cont        hue                                    cat vs cont
cont        hue                                    cont vs cat
cont        hue                                    cat vs cat
---------------------------------------------------------------------------
cont        cont (plotted only when there is cont) cont vs cont
cont        cont (plotted only when there is cont) cat vs cont
cont        cont (plotted only when there is cont) cont vs cat
cont        cont (plotted only when there is cont) cat vs cat

Here we only consder the first four cases
"""

In [ ]:
sns.set(rc={'figure.figsize': (5, 5)})

# Randomly sample 100 points to increase plotting speed
# Or a better way is to randomly sample 10% of the data and plot
df_curr = df.sample(frac=1, random_state=0).iloc[:100, :]

g = sns.PairGrid(df_curr, x_vars=cont_cols, y_vars=cont_cols, hue='price')
# g = sns.PairGrid(df_curr, x_vars=cont_cols, y_vars=cont_cols)
g.map_diag(sns.kdeplot)
g.map_lower(sns.scatterplot)
g.map_upper(sns.regplot)
g.add_legend()

In [ ]:
sns.set(rc={'figure.figsize': (5, 5)})

# Randomly sample 100 points to increase plotting speed
df_curr = df.sample(frac=1, random_state=0).iloc[:100, :]

g = sns.PairGrid(df_curr, x_vars=cat_cols, y_vars=cont_cols, hue='price')
g.map_offdiag(sns.boxplot)  # There is no "diag" when x_vars != y_vars!
g.add_legend()

In [ ]:
sns.set(rc={'figure.figsize': (5, 5)})

# Randomly sample 100 points to increase plotting speed
df_curr = df.sample(frac=1, random_state=0).iloc[:100, :]

g = sns.PairGrid(df_curr, x_vars=cat_cols, y_vars=cat_cols, hue='price')
g.map_diag(sns.countplot)
g.add_legend()

In [ ]:
sns.set(rc={'figure.figsize': (5, 5)})

# Randomly sample 100 points to increase plotting speed
df_curr = df.sample(frac=1, random_state=0).iloc[:100, :]

# pairplot is a high-level interface for PairGrid that is intended to make it easy to draw a few common styles
# You should use PairGrid directly if you need more flexibility.
# It will simply skip the categorical variables, which is bad!
sns.pairplot(df_curr, hue='price')

## Correlations between every pair of variables

In [ ]:
"""
    Variable1                        Variable2                       Correlation
(1) continuous (int or float)        continuous (int or float)       Pearson's correlation (ss.pearsonr(cont1, cont2) or you can directly use df.corr())
(2) continuous (int or float)        category (ordered or not)       Kruskal-Wallis H-test (kruskal_wallis(cat1, cont2))
(3) category (ordered or not)        category (ordered or not)       Chi square statistic (chi_square(cat1, cat2)) / mutual information (mutual_info_score(cat1, cat2))

If you observe very high correlation between some features, consider to drop one of these highly correlated features 
"""

def chi_square(cat1, cat2, use_pvalue=True):
    confusion_table = pd.crosstab(cat1, cat2)
    return ss.chi2_contingency(confusion_table)[1] if use_pvalue else ss.chi2_contingency(confusion_table)[0]

def kruskal_wallis(cat1, cont2, use_pvalue=True):
    temp = pd.concat((cat1, cont2), axis=1)
    samples = tuple(temp.loc[temp[cat1.name] == val, cont2.name].tolist() for val in cat1.unique())
    return ss.kruskal(*samples)[1] if use_pvalue else ss.kruskal(*samples)[0]

def get_corr(df, cols1, cols2):        
    result = pd.DataFrame(index=cols1, columns=cols2)
    for col1 in cols1: 
        for col2 in cols2:
            mask = (df[col1].notnull() & df[col2].notnull())
            c1, c2 = df.loc[mask, col1], df.loc[mask, col2]
            is_cat1, is_cat2 = (c1.dtype.name == 'category'), (c2.dtype.name == 'category')
            if is_cat1 and is_cat2:
                # result.loc[col1, col2] = mutual_info_score(c1, c2)
                result.loc[col1, col2] = chi_square(c1, c2)
            elif is_cat1 and not is_cat2:
                result.loc[col1, col2] = kruskal_wallis(c1, c2)
            elif not is_cat1 and is_cat2:
                result.loc[col1, col2] = kruskal_wallis(c2, c1)
            else:
                result.loc[col1, col2] = ss.pearsonr(c1, c2)[0]
    return result.astype('float').round(5)  # Must make them float to use sns.heatmap; otherwise they are np.float64, which are not accepted by sns.heatmap

In [ ]:
sns.set(rc={'figure.figsize': (20, 20)})

cont_cont_corr = get_corr(df, cont_cols, cont_cols)
# cont_cont_corr should be the same as df.corr(), as df.corr() will skip / ignore any non-continuous variables
assert (cont_cont_corr - df.corr()).abs().max().max() < 1e-5
sns.heatmap(cont_cont_corr, cmap='Reds', annot=True)
cont_cont_corr

In [ ]:
sns.set(rc={'figure.figsize': (20, 20)})

cat_cont_corr = get_corr(df, cat_cols, cont_cols)
sns.heatmap(cat_cont_corr, cmap='Reds', annot=True)
cat_cont_corr

In [ ]:
sns.set(rc={'figure.figsize': (20, 20)})

cat_cat_corr = get_corr(df, cat_cols, cat_cols)
sns.heatmap(cat_cat_corr, cmap='Reds', annot=True)
cat_cat_corr

# Training and testing split

## Deduplication

In [ ]:
# We check whether there is duplication to avoid leaking data to the testing set
df.loc[df.duplicated(keep=False), :]

## Split

In [ ]:
x_df = df.loc[:, cat_cols + cont_cols]
y_df = df.loc[:, res_cols]

# Use stratify to make sure the label distributions in training and testing sets are the same
# See https://gist.github.com/SHi-ON/63839f3a3647051a180cb03af0f7d0d9
# We use the while loop to ensure all categories of all categorical varialbes occur in the training set
seed = -1
while seed == -1 or any(x_train[col].nunique() != df[col].nunique() for col in cat_cols):
    seed += 1
    x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.3, random_state=seed, stratify=y_df)
    
print(seed)
print(x_train.shape)
print(x_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

# Feature engineering

## Outlier detection and handling

### Consider each individual dimension

In [ ]:
"""
First, you should understand we only need to handle outliers for continuous variables

The main method to detect outliers for each individual dimension is to plot boxplot, histplot and kdeplot

You may observe three patterns when you look at the histplot:

(1) There is a peak on one side, there are very few values far away from the peak, and there is no density in the middle 
    -> The far-away points are outliers 
    -> You can 
       1) discard them
       2) truncate them: for any values out of 2.5% quantile (or Q1 - 1.5IQR) and 97.5% quantile (or Q3 + 1.5IQR), we let them be 2.5% quantile (or Q1 - 1.5IQR) or 97.5% quantile (or Q3 + 1.5IQR)
       3) let them be NA, which will be imputed in the following missing value imputation step
    -> You may also consider to tell engineers there may be something wrong with their logging code
       
(2) There is a peak on one side, there are very few values far away from the peak, but there is some density in the middle 
    -> The distribution is highly skewed (in this case, you shouldn't discard or truncate the values; otherwise you lose lots of information)
    -> You can
       1) do the log transformation if all values > 0: x -> log(x)
       2) do the plus1-log transformation if all values >= 0: x -> log(x + 1) (0 will still be converted to 0)
       3) make x be x ^ (1 / 3) if some values < 0

(3) There is a peak on one side, there are lots of values far away from the peak (i.e., it is another peak)
    -> The distribution is multi-mode (in this case, you shouldn't discard or truncate the values; otherwise you lose lots of information)
    -> There is nothing you can do. Just leave it as it is
"""

In [ ]:
fig, axes = plt.subplots(len(cont_cols), 3, figsize=(30, len(cont_cols) * 7))

fig.suptitle('Plots to detect outliers for each individual continuous variable')
fig.tight_layout()
fig.subplots_adjust(top=0.95)

for i, col in enumerate(cont_cols):
    sns.boxplot(ax=axes[i][0], x=col, data=x_train)
    sns.histplot(ax=axes[i][1], x=col, data=x_train)
    sns.kdeplot(ax=axes[i][2], x=col, data=x_train, common_norm=True)

In [ ]:
"""
https://www.quora.com/Should-we-exclude-outliers-form-testing-set
If you are trying to test how good your model does with outliers in your data (maybe in real-world you won’t be able to curate the data to reduce outliers) then you should include the outliers in your test data. 
But, if you are confident that you would be able to filter outliers before it comes to your machine learning model for prediction, there is no point having the outliers in your test data.
"""

class OutlierClipper:
    def __init__(self, method):
        self.method = method
        self.bounds = dict()
        
    def fit(self, df):
        assert len(self.bounds) == 0
        
        # Make sure all columns of df are continuous variables
        for col in df.columns:
            if self.method == 'iqr':
                # See https://www.analyticsvidhya.com/blog/2021/05/feature-engineering-how-to-detect-and-remove-outliers-with-python-code/
                q25 = df[col].quantile(0.25)
                q75 = df[col].quantile(0.75)
                iqr = q75 - q25
                lb = q25 - 1.5 * iqr
                ub = q75 + 1.5 * iqr
            elif type(self.method) == tuple:
                # Winsorizing: https://en.wikipedia.org/wiki/Winsorizing
                lb = df[col].quantile(self.method[0])
                ub = df[col].quantile(self.method[1])
            else:
                raise NotImplemented
            self.bounds[col] = (lb, ub)
            
    def transform(self, df):
        assert len(self.bounds) != 0 and sorted(self.bounds.keys()) == sorted(df.columns)
        
        df_result = df.copy()
        for col in df.columns:
            lb, ub = self.bounds[col]
            df_result[col] = df[col].clip(lb, ub)
            print('{0:d} outliers for {1:} clipped'.format((~df[col].between(lb, ub)).sum(), col))
        
        return df_result
    
# The large values in these two columns actually are not large enough to be regarded as "outliers"
# If the large values are like "100", then they can be regarded as outliers
# Here, since we want to try the OutlierClipper, we simply regard them as outliers
clip_cols = ['bedrooms', 'bathrooms']

outlier_clipper = OutlierClipper(method=(0.005, 0.995))
outlier_clipper.fit(x_train.loc[:, clip_cols])
x_train.loc[:, clip_cols] = outlier_clipper.transform(x_train.loc[:, clip_cols])
x_test.loc[:, clip_cols] = outlier_clipper.transform(x_test.loc[:, clip_cols])

In [ ]:
class LogTransformer:
    def __init__(self):
        self.min_val = dict()
        
    def fit(self, df):
        assert len(self.min_val) == 0        
        self.min_val = {col: df[col].min() for col in df.columns}

    def transform(self, df):
        assert len(self.min_val) != 0 and sorted(self.min_val.keys()) == sorted(df.columns)
        
        df_result = df.copy()
        for col in df.columns:
            min_val = self.min_val[col]
            if min_val <= 0:
                df_result[col] = df[col].map(lambda x: np.log(x - min_val + 1))
            else:
                df_result[col] = df[col].map(lambda x: np.log(x))
        
        return df_result

log_cols = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15']
log_transformer = LogTransformer()
log_transformer.fit(x_train.loc[:, log_cols])
x_train.loc[:, log_cols] = log_transformer.transform(x_train.loc[:, log_cols])
x_test.loc[:, log_cols] = log_transformer.transform(x_test.loc[:, log_cols])

In [ ]:
fig, axes = plt.subplots(len(cont_cols), 3, figsize=(30, len(cont_cols) * 7))

fig.suptitle('Plots for each individual continuous variable after handling outliers')
fig.tight_layout()
fig.subplots_adjust(top=0.95)

for i, col in enumerate(cont_cols):
    sns.boxplot(ax=axes[i][0], x=col, data=x_train)
    sns.histplot(ax=axes[i][1], x=col, data=x_train)
    sns.kdeplot(ax=axes[i][2], x=col, data=x_train, common_norm=True)

### Consider all dimensions (optional)

In [ ]:
# See https://scikit-learn.org/stable/modules/outlier_detection.html
# See https://github.com/yzhao062/pyod

df_curr = x_train.loc[x_train.notnull().all(axis=1), cont_cols]  # The following outlier detectors can't handle nan values, so we remove all lines with >= 1 nan

outlier_detector = LocalOutlierFactor(n_neighbors=20)
outlier_detector.fit_predict(df_curr)

# A negative_outlier_factor_ close to -1 means a inliers, while a smaller negative_outlier_factor_ means an outlier
outlier_score = pd.Series(outlier_detector.negative_outlier_factor_, index=df_curr.index)
outlier_score.name = 'outlier_score'
temp = pd.concat((df_curr, outlier_score), axis=1)

sns.histplot(x='outlier_score', data=temp)

# According to the histogram, the samples with negative_outlier_factor_ <= -2 could be outliers
temp.loc[temp['outlier_score'] <= -2, :].sort_values(by=['outlier_score', 'sqft_living'])

In [ ]:
df_curr = x_train.loc[x_train.notnull().all(axis=1), cont_cols]

outlier_detector = IsolationForest(random_state=0)
outlier_label = outlier_detector.fit_predict(df_curr)

outlier_label = pd.Series(outlier_label, index=df_curr.index)
outlier_label.name = 'outlier_label'
temp = pd.concat((df_curr, outlier_label), axis=1)

temp.loc[temp['outlier_label'] == -1, :].sort_values(by='sqft_living')

## Missing value

In [ ]:
"""
Here you may want to ask them why these data are missing

(1) The data could be Missing Completely at Random (MCAR), i.e., the events that lead to the value being missing doesn't depend on observed or unobserved values
(2) The data could be Missing at Random (MAR), i.e., the events that lead to the value being missing depends only on observed values
(3) The data could be Missing not at Random (MNAR), i.e., the events that lead to the value being missing depends on unobserved values 
    (e.g., Tom's exam score is missing because he hides the exam solution book, as he got a low score and he doesn't want other people to know it, i.e., 
     the reason it is missing is related to the unobserved value of his exam score)
     
    Another example of MNAR. When you do random sampling, some item is not observed (its count is zero), 
    because the total count of this item in the population is too small and a simple random sampling can't capture it
    In this case, we may impute its count by "min count of the observed items / sqrt(2)"
"""

In [ ]:
# See https://scikit-learn.org/stable/modules/impute.html

# Univariate feature imputation for categorical variables

# We must convert categorical variables to float first, otherwise SimpleImputer can't correctly handle it
for col in cat_cols:
    x_train[col] = x_train[col].astype(float)
cat_imp = SimpleImputer(strategy='most_frequent')
cat_imp.fit(x_train.loc[:, cat_cols])
x_train.loc[:, cat_cols] = cat_imp.transform(x_train.loc[:, cat_cols])
for col in cat_cols:
    x_train[col] = pd.Categorical(x_train[col], ordered=True)
    
for col in cat_cols:
    x_test[col] = x_test[col].astype(float)
x_test.loc[:, cat_cols] = cat_imp.transform(x_test.loc[:, cat_cols])
for col in cat_cols:
    x_test[col] = pd.Categorical(x_test[col], ordered=True)

In [ ]:
# Univariate feature imputation for continuous variables

cont_imp = SimpleImputer(strategy='mean')
cont_imp.fit(x_train.loc[:, cont_cols])
x_train.loc[:, cont_cols] = cont_imp.transform(x_train.loc[:, cont_cols])
x_test.loc[:, cont_cols] = cont_imp.transform(x_test.loc[:, cont_cols])

In [ ]:
# Multivariate feature imputation for continuous variables

# Can only be used for continuous variables
iter_imp = IterativeImputer(max_iter=10, random_state=0)
iter_imp.fit(x_train.loc[:, cont_cols])
x_train.loc[:, cont_cols] = iter_imp.transform(x_train.loc[:, cont_cols])
x_test.loc[:, cont_cols] = iter_imp.transform(x_test.loc[:, cont_cols])

## Standardization

In [ ]:
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(x_train.loc[:, cont_cols])
x_train.loc[:, cont_cols] = scaler.transform(x_train.loc[:, cont_cols])
x_test.loc[:, cont_cols] = scaler.transform(x_test.loc[:, cont_cols])

In [ ]:
fig, axes = plt.subplots(len(cont_cols), 3, figsize=(30, len(cont_cols) * 7))

fig.suptitle('Plots for each individual continuous variable after handling outliers')
fig.tight_layout()
fig.subplots_adjust(top=0.95)

for i, col in enumerate(cont_cols):
    sns.boxplot(ax=axes[i][0], x=col, data=x_train)
    sns.histplot(ax=axes[i][1], x=col, data=x_train)
    sns.kdeplot(ax=axes[i][2], x=col, data=x_train, common_norm=True)

## One-hot encoding

In [ ]:
# We append x_test to x_train and call pd.get_dummies, and then we separate them back
# We use this trick to handle the cases: (1) there are categories that are in x_train but are not in x_test; (2) there are categories that are in x_test but are not in x_train
# Without this trick, the one-hot vectors in x_train and x_test could mismatch

assert (x_train.columns == x_test.columns).all()
x_train_test = x_train.append(x_test)
x_train_test_dummies = pd.get_dummies(x_train_test, columns=cat_cols, drop_first=True)

x_train = x_train_test_dummies.iloc[:x_train.shape[0], :]
x_test = x_train_test_dummies.iloc[x_train.shape[0]:, :]

In [ ]:
x_train

In [ ]:
x_test

## PCA (optional)

In [ ]:
# PCA
pca = PCA(n_components=2)
pca.fit(x_train.loc[:, cont_cols])
x_train_pca = pd.DataFrame(pca.transform(x_train.loc[:, cont_cols]))

sns.scatterplot(x=0, y=1, data=x_train_pca)
# for i in x_train_pca.index:
#     plt.annotate(i, (x_train_pca.loc[i, 0] + 0.7, x_train_pca.loc[i, 1] + 0.7) )
    
x_train_pca

# Model training and evaluation

## Logistic regression + binary classification

### Load training and testing data

In [ ]:
folder = 'bin_clf_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# Train the model once
"""
'newton-cg' - ['l2', 'none']
'lbfgs' - ['l2', 'none']
'liblinear' - ['l1', 'l2']
'sag' - ['l2', 'none']
'saga' - ['elasticnet', 'l1', 'l2', 'none']
"""

model = LogisticRegression(penalty='l1', C=1.0, fit_intercept=True, class_weight=None, 
                           random_state=0, solver='liblinear', max_iter=100, multi_class='ovr').fit(x_train, y_train)

print(model.score(x_train, y_train))
model.coef_

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'penalty': ['l1', 'l2'], 'solver': ['liblinear'], 'C': [100, 10, 5, 2, 1], 'class_weight': [{0: 1.2, 1: 1.0}, 'balanced']},
    {'penalty': ['none'], 'solver': ['newton-cg'], 'C': [100, 10, 5, 2, 1], 'class_weight': [{0: 1.2, 1: 1.0}, 'balanced']}
]

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, pos_label=1, average='binary')
recall_scorer = make_scorer(recall_score, pos_label=1, average='binary')
scoring = {'AUC': 'roc_auc', 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

gs = GridSearchCV(
    LogisticRegression(random_state=0),
    param_grid=param_grid,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1
)
gs.fit(x_train, y_train)

# rs = RandomizedSearchCV(
#     LogisticRegression(random_state=0),
#     param_distributions=param_grid,
#     n_iter=15,
#     scoring=scoring,
#     refit='AUC',
#     return_train_score=False,
#     cv=5, n_jobs=-1, random_state=0
# )
# rs.fit(x_train, y_train)

model = gs.best_estimator_
print(gs.best_estimator_)
print(gs.best_params_)

result = gs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=2)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=2)

In [ ]:
plot_roc_curve(model, x_test, y_test)

In [ ]:
plot_precision_recall_curve(model, x_test, y_test)

## Random forest + binary classification

### Load training and testing data

In [ ]:
folder = 'bin_clf_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# Train the model once
model = RandomForestClassifier(n_estimators=100, 
                               criterion='gini',
                               max_depth=3, # Avoid overfitting by controling the complexity of the tree
                               max_leaf_nodes=None, # Avoid overfitting by controling the complexity of the tree
                               ccp_alpha=0.0, # Avoid overfitting by pruning the tree
                               min_samples_split=2, # Avoid overfitting by avoid splitting too much
                               min_samples_leaf=1, # Avoid overfitting by avoid splitting too much
                               min_weight_fraction_leaf=0.0, # Avoid overfitting by avoid splitting too much
                               min_impurity_decrease=0.0, # Avoid overfitting by avoid splitting too much
                               max_features='sqrt', # Make sure trees are independent
                               bootstrap=True, # Make sure trees are independent
                               max_samples=None, # Make sure trees are independent
                               oob_score=True,
                               class_weight='balanced',
                               n_jobs=-1,
                               random_state=0).fit(x_train, y_train)

print(model.score(x_train, y_train))
tree.plot_tree(model.estimators_[0], feature_names=x_train.columns)  # Visualize one of the trees in the random forest
plt.show()

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'n_estimators': [100, 200], 
     'criterion': ['gini', 'entropy'], 
     'max_depth': [3, 6], 
     'min_samples_split': [2, 10],
     'min_samples_leaf': [1, 5],
     'min_impurity_decrease': [0, 0.02],
     'class_weight': [{0: 1.2, 1: 1.0}, 'balanced']}
]

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, pos_label=1, average='binary')
recall_scorer = make_scorer(recall_score, pos_label=1, average='binary')
scoring = {'AUC': 'roc_auc', 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

# gs = GridSearchCV(
#     RandomForestClassifier(random_state=0),
#     param_grid=param_grid,
#     scoring=scoring,
#     refit='AUC',
#     return_train_score=False,
#     cv=5, n_jobs=-1
# )
# gs.fit(x_train, y_train)

rs = RandomizedSearchCV(
    RandomForestClassifier(random_state=0),
    param_distributions=param_grid,
    n_iter=30,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=2)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=2)

In [ ]:
plot_roc_curve(model, x_test, y_test)

In [ ]:
plot_precision_recall_curve(model, x_test, y_test)

## GBDT + binary classification

### Load training and testing data

In [ ]:
folder = 'bin_clf_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
"""
To understand how to use GBDT for classification, see
https://projecteuclid.org/journals/annals-of-statistics/volume-29/issue-5/Greedy-function-approximation-A-gradient-boosting-machine/10.1214/aos/1013203451.full
https://stats.stackexchange.com/questions/204154/classification-with-gradient-boosting-how-to-keep-the-prediction-in-0-1
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

For binary classification, GBDT is trying to use a regression tree to model the log odds ratio (the "linear" part), instead of the observed 0, 1
For multiple classification with K classes, GBDT will build K trees at the same time to model the "linear" part of the probability (created by softmax) of each class 
"""

# Train the model once
model = GradientBoostingClassifier(loss='deviance',  # binomial or multinomial deviance loss function
                                   learning_rate=0.1, # Learning rate shrinks the contribution of each tree by learning_rate. The larger the learning rate is, the fewer estimators we need
                                   n_estimators=100, # Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance
                                   criterion='friedman_mse',
                                   init='zero', # An estimator object that is used to compute the initial predictions
                                   max_depth=3, # Avoid overfitting by controling the complexity of the tree
                                   max_leaf_nodes=None, # Avoid overfitting by controling the complexity of the tree
                                   ccp_alpha=0.0, # Avoid overfitting by pruning the tree
                                   min_samples_split=2, # Avoid overfitting by avoid splitting too much
                                   min_samples_leaf=1, # Avoid overfitting by avoid splitting too much
                                   min_weight_fraction_leaf=0.0, # Avoid overfitting by avoid splitting too much
                                   min_impurity_decrease=0.0, # Avoid overfitting by avoid splitting too much
                                   subsample=1.0,  # The fraction of samples to be used for fitting the individual base learners. If smaller than 1.0 this results in Stochastic Gradient Boosting. subsample interacts with the parameter n_estimators. Choosing subsample < 1.0 leads to a reduction of variance and an increase in bias
                                   max_features=None, # Make sure trees are independent
                                   random_state=0).fit(x_train, y_train)  # Unlike Random Forest, there is no "class_weight" in model, we only have "sample_weight" in fit (see https://scikit-learn.org/0.24/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier.fit)

print(model.score(x_train, y_train))
tree.plot_tree(model.estimators_[0][0], feature_names=x_train.columns)  # Visualize one of the trees in the random forest, where now model.estimators_ is an array of size n_estimators * loss_.K
plt.show()

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'learning_rate': [0.01, 0.1],
     'n_estimators': [50, 100], 
     'max_depth': [3, 6], 
     'min_samples_split': [2, 10],
     'min_samples_leaf': [1, 5],
     'min_impurity_decrease': [0, 0.02], 
     'subsample': [1, 0.8]}
]

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, pos_label=1, average='binary')
recall_scorer = make_scorer(recall_score, pos_label=1, average='binary')
scoring = {'AUC': 'roc_auc', 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=0),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=2)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=2)

In [ ]:
plot_roc_curve(model, x_test, y_test)

In [ ]:
plot_precision_recall_curve(model, x_test, y_test)

## XGBoost + binary classification

### Load training and testing data

In [ ]:
folder = 'bin_clf_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# Reference: 
# https://xgboost.readthedocs.io/en/latest/parameter.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=feature_importances#xgboost.XGBClassifier
# https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html

"""
You may save a trained model by: 
model.save_model('/path/to/model.json')

You may load the saved model by:
model = xgb.XGBClassifier()
model.load_model('/path/to/model.json')
"""

model = xgb.XGBClassifier(
        n_estimators=4,
        max_depth=3,
        learning_rate=0.9,
        objective='binary:logistic',
        booster='gbtree',
        tree_method='exact',
        gamma=0.0,
        min_child_weight=1.0,
        max_delta_step=0.0,
        reg_alpha=0.0,
        reg_lambda=0.5,
        random_state=0,
        use_label_encoder=False,
        missing=np.nan,
        importance_type=None,
)

"""
If "balanced", class weight for class i = (n_total_samples / n_classes) * (1 / n_samples_of_class_i)
If a dictionary is given, keys are classes and values are corresponding class weights
If None is given, the class weights will be uniform
"""
weights = compute_class_weight('balanced', classes=y_train.unique(), y=y_train)
dict_weights = dict(zip(y_train.unique(), weights))
print(dict_weights)

model.fit(x_train, y_train, eval_metric='error', sample_weight=y_train.replace(dict_weights))

print(model.score(x_train, y_train))

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'n_estimators': [5, 10, 15, 20], 
     'learning_rate': [0.3, 0.5, 0.7, 0.9, 1.0],
     'max_depth': [3, 6, 9, 12], 
     'gamma': [0.0, 0.5, 1.0],
     'min_child_weight': [1.0, 2.0],
     'max_delta_step': [0.0, 0.5], 
     'lambda': [0.0, 0.5, 1.0], 
     'alpha': [0.0, 0.5, 1.0], 
    }
]

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, pos_label=1, average='binary')
recall_scorer = make_scorer(recall_score, pos_label=1, average='binary')
scoring = {'AUC': 'roc_auc', 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    xgb.XGBClassifier(use_label_encoder=False, random_state=0),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
weights = compute_class_weight('balanced', classes=y_train.unique(), y=y_train)
dict_weights = dict(zip(y_train.unique(), weights))
rs.fit(x_train, y_train, eval_metric='error', sample_weight=y_train.replace(dict_weights))

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=2)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=2)

In [ ]:
plot_roc_curve(model, x_test, y_test)

In [ ]:
plot_precision_recall_curve(model, x_test, y_test)

## MLP + binary classification

### Load training and testing data

In [ ]:
folder = 'bin_clf_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

model = MLPClassifier(
    hidden_layer_sizes=(30, 20, 10),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size=256,
    learning_rate='constant', # Only used when solver='sgd'
    learning_rate_init=0.001, # Only used when solver='sgd' or 'adam'
    power_t=0.5, # It is used in updating effective learning rate when the learning_rate is set to 'invscaling'. Only used when solver='sgd'
    max_iter=200, # For stochastic solvers ('sgd', 'adam'), note that this determines the number of epochs (how many times each data point will be used), not the number of gradient steps
    shuffle=True, # Whether to shuffle samples in each iteration. Only used when solver='sgd' or 'adam'
    random_state=0,
    early_stopping=False,
    validation_fraction=0.1 # The proportion of training data to set aside as validation set for early stopping. Must be between 0 and 1. Only used if early_stopping is True
).fit(x_train, y_train)

print(model.score(x_train, y_train))

print(model.n_layers_)
print(model.n_outputs_)
print(model.out_activation_)

In [ ]:
df_loss = pd.DataFrame({'n_epoch': list(range(1, len(model.loss_curve_) + 1)), 'train_loss': model.loss_curve_})
sns.lineplot(x='n_epoch', y='train_loss', data=df_loss)

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'hidden_layer_sizes': list(itertools.product(*[[40, 30], [30, 20], [20, 10, 5]])), 
     'alpha': [0.001, 0.0001], 
     'batch_size': [512, 256], 
     'learning_rate_init': [0.01, 0.001]},
]
"""
Candidate values of hidden_layer_sizes are
[(40, 30, 20),
 (40, 30, 10),
 (40, 30, 5),
 (40, 20, 20),
 (40, 20, 10),
 (40, 20, 5),
 (30, 30, 20),
 (30, 30, 10),
 (30, 30, 5),
 (30, 20, 20),
 (30, 20, 10),
 (30, 20, 5)]
"""

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, pos_label=1, average='binary')
recall_scorer = make_scorer(recall_score, pos_label=1, average='binary')
scoring = {'AUC': 'roc_auc', 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    MLPClassifier(random_state=0),
    param_distributions=param_grid,
    n_iter=5,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=2)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=2)

In [ ]:
plot_roc_curve(model, x_test, y_test)

In [ ]:
plot_precision_recall_curve(model, x_test, y_test)

## SVC + binary classification

### Load training and testing data

In [ ]:
folder = 'bin_clf_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
model = SVC(C=1.0, 
            kernel='rbf',
            degree=3, # Degree of the polynomial kernel function ('poly'). Ignored by all other kernels
            gamma='auto', # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'; if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma; if 'auto', uses 1 / n_features
            coef0=0.0, # Independent term in kernel function. It is only significant in 'poly' and 'sigmoid'
            shrinking=True, 
            probability=False, # Whether to enable probability estimates. This must be enabled prior to calling fit, will slow down that method as it internally uses 5-fold cross-validation
            class_weight='balanced',
            random_state=0).fit(x_train, y_train)

print(model.score(x_train, y_train))
model.support_vectors_

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'C': [100, 10, 1], 
     'kernel': ['linear', 'rbf', 'poly'], 
     'class_weight': [{0: 1.2, 1: 1.0}, 'balanced']},
]

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, pos_label=1, average='binary')
recall_scorer = make_scorer(recall_score, pos_label=1, average='binary')
scoring = {'AUC': 'roc_auc', 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    SVC(random_state=0),
    param_distributions=param_grid,
    n_iter=5,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
# proba_pred = model.predict_proba(x_test)  # predict_proba is not available when probability=False
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

# assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=2)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=2)

In [ ]:
plot_roc_curve(model, x_test, y_test)

In [ ]:
plot_precision_recall_curve(model, x_test, y_test)

## Logistic regression + multiple classification

### Load training and testing data

In [ ]:
folder = 'mul_clf_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# Train the model once
model = LogisticRegression(penalty='l1', C=1.0, fit_intercept=True, class_weight=None, 
                           random_state=0, solver='saga', tol=5e-3, max_iter=200, multi_class='multinomial').fit(x_train, y_train)

print(model.score(x_train, y_train))
model.coef_

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'penalty': ['l1'], 'solver': ['saga'], 'C': [100, 10, 5, 2, 1], 'class_weight': [{0: 1.2, 1: 1.0, 2: 2.0}, 'balanced']},
    {'penalty': ['l2', 'none'], 'solver': ['newton-cg'], 'C': [100, 10, 5, 2, 1], 'class_weight': [{0: 1.2, 1: 1.0, 2: 2.0}, 'balanced']}
]

def precision_2(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[2]
def recall_2(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[2]
def multiclass_auc(y_true, y_prob):
    return roc_auc_score(y_true, y_prob, multi_class='ovo', average='macro')
auc_scorer = make_scorer(multiclass_auc, needs_proba=True, greater_is_better=True)  # multiclass_auc requires to accept y_prob as input, so we need to set needs_proba=True
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)
precision_scorer = make_scorer(precision_2, greater_is_better=True)
recall_scorer = make_scorer(recall_2, greater_is_better=True)
scoring = {'AUC': auc_scorer, 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    LogisticRegression(tol=5e-3, max_iter=200, multi_class='multinomial', random_state=0),
    param_distributions=param_grid,
    n_iter=15,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=3)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1', 'class 2']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=3)

## Random forest + multiple classification

### Load training and testing data

In [ ]:
folder = 'mul_clf_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# Train the model once
model = RandomForestClassifier(n_estimators=100, 
                               criterion='gini',
                               max_depth=3, # Avoid overfitting by controling the complexity of the tree
                               max_leaf_nodes=None, # Avoid overfitting by controling the complexity of the tree
                               ccp_alpha=0.0, # Avoid overfitting by pruning the tree
                               min_samples_split=2, # Avoid overfitting by avoid splitting too much
                               min_samples_leaf=1, # Avoid overfitting by avoid splitting too much
                               min_weight_fraction_leaf=0.0, # Avoid overfitting by avoid splitting too much
                               min_impurity_decrease=0.0, # Avoid overfitting by avoid splitting too much
                               max_features='sqrt', # Make sure trees are independent
                               bootstrap=False, # Make sure trees are independent
                               max_samples=None, # Make sure trees are independent
                               oob_score=False,
                               class_weight=None,
                               n_jobs=-1,
                               random_state=0).fit(x_train, y_train)

print(model.score(x_train, y_train))
# When boostrap is True, samples may not be equal to the sum of values in each node, as "samples" is the "unique" number of samples, but "value" is the number of "repeated" samples
# https://stackoverflow.com/questions/56103507/why-does-this-decision-trees-values-at-each-step-not-sum-to-the-number-of-sampl
plt.rcParams["figure.figsize"] = (60, 60)
tree.plot_tree(model.estimators_[0], feature_names=x_train.columns)  # Visualize one of the trees in the random forest
plt.show()

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'n_estimators': [100, 200], 
     'criterion': ['gini', 'entropy'], 
     'max_depth': [3, 6], 
     'min_samples_split': [2, 10],
     'min_samples_leaf': [1, 5],
     'min_impurity_decrease': [0, 0.02],
     'class_weight': [{0: 1.2, 1: 1.0, 2: 1.0}, 'balanced']}
]

def precision_2(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[2]
def recall_2(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[2]
def multiclass_auc(y_true, y_prob):
    return roc_auc_score(y_true, y_prob, multi_class='ovo', average='macro')
auc_scorer = make_scorer(multiclass_auc, needs_proba=True, greater_is_better=True)  # multiclass_auc requires to accept y_prob as input, so we need to set needs_proba=True
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)
precision_scorer = make_scorer(precision_2, greater_is_better=True)
recall_scorer = make_scorer(recall_2, greater_is_better=True)
scoring = {'AUC': auc_scorer, 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    RandomForestClassifier(random_state=0),
    param_distributions=param_grid,
    n_iter=30,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=3)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1', 'class 2']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=3)

## GBDT + multiple classification

### Load training and testing data

In [ ]:
folder = 'mul_clf_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# Train the model once
model = GradientBoostingClassifier(loss='deviance',  # binomial or multinomial deviance loss function
                                   learning_rate=0.1, # Learning rate shrinks the contribution of each tree by learning_rate. The larger the learning rate is, the fewer estimators we need
                                   n_estimators=100, # Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance
                                   criterion='friedman_mse',
                                   init='zero', # An estimator object that is used to compute the initial predictions
                                   max_depth=3, # Avoid overfitting by controling the complexity of the tree
                                   max_leaf_nodes=None, # Avoid overfitting by controling the complexity of the tree
                                   ccp_alpha=0.0, # Avoid overfitting by pruning the tree
                                   min_samples_split=2, # Avoid overfitting by avoid splitting too much
                                   min_samples_leaf=1, # Avoid overfitting by avoid splitting too much
                                   min_weight_fraction_leaf=0.0, # Avoid overfitting by avoid splitting too much
                                   min_impurity_decrease=0.0, # Avoid overfitting by avoid splitting too much
                                   subsample=1.0,  # The fraction of samples to be used for fitting the individual base learners. If smaller than 1.0 this results in Stochastic Gradient Boosting. subsample interacts with the parameter n_estimators. Choosing subsample < 1.0 leads to a reduction of variance and an increase in bias
                                   max_features=None, # Make sure trees are independent
                                   random_state=0).fit(x_train, y_train)

print(model.score(x_train, y_train))
tree.plot_tree(model.estimators_[0][0], feature_names=x_train.columns)  # Visualize one of the trees in the random forest, where now model.estimators_ is an array of size n_estimators * loss_.K
plt.show()

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'learning_rate': [0.01, 0.1],
     'n_estimators': [50, 100], 
     'max_depth': [3, 6], 
     'min_samples_split': [2, 10],
     'min_samples_leaf': [1, 5],
     'min_impurity_decrease': [0, 0.02], 
     'subsample': [1, 0.8]}
]

def precision_2(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[2]
def recall_2(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[2]
def multiclass_auc(y_true, y_prob):
    return roc_auc_score(y_true, y_prob, multi_class='ovo', average='macro')
auc_scorer = make_scorer(multiclass_auc, needs_proba=True, greater_is_better=True)  # multiclass_auc requires to accept y_prob as input, so we need to set needs_proba=True
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)
precision_scorer = make_scorer(precision_2, greater_is_better=True)
recall_scorer = make_scorer(recall_2, greater_is_better=True)
scoring = {'AUC': auc_scorer, 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=0),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=3)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1', 'class 2']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=3)

## XGBoost + multiple classification

### Load training and testing data

In [ ]:
folder = 'mul_clf_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# Reference: 
# https://xgboost.readthedocs.io/en/latest/parameter.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=feature_importances#xgboost.XGBClassifier
# https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html

model = xgb.XGBClassifier(
        n_estimators=4,
        max_depth=3,
        learning_rate=0.9,
        objective='multi:softprob',
        booster='gbtree',
        tree_method='exact',
        gamma=0.0,
        min_child_weight=1.0,
        max_delta_step=0.0,
        reg_alpha=0.0,
        reg_lambda=0.5,
        random_state=0,
        use_label_encoder=False,
        missing=np.nan,
        importance_type=None,
        # num_class=y_train.nunique()  # You don't have to manually specify the number of classes here
)

"""
If "balanced", class weight for class i = (n_total_samples / n_classes) * (1 / n_samples_of_class_i)
If a dictionary is given, keys are classes and values are corresponding class weights
If None is given, the class weights will be uniform
"""
weights = compute_class_weight('balanced', classes=y_train.unique(), y=y_train)
dict_weights = dict(zip(y_train.unique(), weights))
print(dict_weights)

model.fit(x_train, y_train, eval_metric='merror', sample_weight=y_train.replace(dict_weights))

print(model.score(x_train, y_train))

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
param_grid = [
    {'n_estimators': [5, 10, 15, 20], 
     'learning_rate': [0.3, 0.5, 0.7, 0.9, 1.0],
     'max_depth': [3, 6, 9, 12], 
     'gamma': [0.0, 0.5, 1.0],
     'min_child_weight': [1.0, 2.0],
     'max_delta_step': [0.0, 0.5], 
     'lambda': [0.0, 0.5, 1.0], 
     'alpha': [0.0, 0.5, 1.0], 
    }
]

def precision_2(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[2]
def recall_2(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[2]
def multiclass_auc(y_true, y_prob):
    return roc_auc_score(y_true, y_prob, multi_class='ovo', average='macro')
auc_scorer = make_scorer(multiclass_auc, needs_proba=True, greater_is_better=True)  # multiclass_auc requires to accept y_prob as input, so we need to set needs_proba=True
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)
precision_scorer = make_scorer(precision_2, greater_is_better=True)
recall_scorer = make_scorer(recall_2, greater_is_better=True)
scoring = {'AUC': auc_scorer, 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    xgb.XGBClassifier(use_label_encoder=False, random_state=0),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
weights = compute_class_weight('balanced', classes=y_train.unique(), y=y_train)
dict_weights = dict(zip(y_train.unique(), weights))
rs.fit(x_train, y_train, eval_metric='merror', sample_weight=y_train.replace(dict_weights))

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=3)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1', 'class 2']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=3)

## MLP + multiple classification

### Load training and testing data

In [ ]:
folder = 'mul_clf_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

model = MLPClassifier(
    hidden_layer_sizes=(30, 20, 10),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size=256,
    learning_rate='constant', # Only used when solver='sgd'
    learning_rate_init=0.001, # Only used when solver='sgd' or 'adam'
    power_t=0.5, # It is used in updating effective learning rate when the learning_rate is set to 'invscaling'. Only used when solver='sgd'
    max_iter=300, # For stochastic solvers ('sgd', 'adam'), note that this determines the number of epochs (how many times each data point will be used), not the number of gradient steps
    shuffle=True, # Whether to shuffle samples in each iteration. Only used when solver='sgd' or 'adam'
    random_state=0,
    early_stopping=False,
    validation_fraction=0.1 # The proportion of training data to set aside as validation set for early stopping. Must be between 0 and 1. Only used if early_stopping is True
).fit(x_train, y_train)

print(model.score(x_train, y_train))

print(model.n_layers_)
print(model.n_outputs_)
print(model.out_activation_)

In [ ]:
df_loss = pd.DataFrame({'n_epoch': list(range(1, len(model.loss_curve_) + 1)), 'train_loss': model.loss_curve_})
sns.lineplot(x='n_epoch', y='train_loss', data=df_loss)

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'hidden_layer_sizes': list(itertools.product(*[[40, 30], [30, 20], [20, 10, 5]])), 
     'alpha': [0.001, 0.0001], 
     'batch_size': [512, 256], 
     'learning_rate_init': [0.01, 0.001]},
]

def precision_2(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[2]
def recall_2(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[2]
def multiclass_auc(y_true, y_prob):
    return roc_auc_score(y_true, y_prob, multi_class='ovo', average='macro')
auc_scorer = make_scorer(multiclass_auc, needs_proba=True, greater_is_better=True)  # multiclass_auc requires to accept y_prob as input, so we need to set needs_proba=True
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)
precision_scorer = make_scorer(precision_2, greater_is_better=True)
recall_scorer = make_scorer(recall_2, greater_is_better=True)
scoring = {'AUC': auc_scorer, 'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    MLPClassifier(max_iter=300, random_state=0),
    param_distributions=param_grid,
    n_iter=5,
    scoring=scoring,
    refit='AUC',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_AUC', 'std_test_AUC', 'mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_AUC', 'mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=3)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1', 'class 2']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=3)

## SVC + multiple classification

### Load training and testing data

In [ ]:
folder = 'mul_clf_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

### Train the model once

In [ ]:
"""
It uses either 'ovo' or 'ovr' to adapt SVC to multi-classification, which can be specified by "decision_function_shape"

See https://scikit-learn.org/stable/modules/svm.html#multi-class-classification
ovo: https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsOneClassifier.html
ovr: https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html
"""

model = SVC(C=1.0, 
            kernel='rbf',
            degree=3, # Degree of the polynomial kernel function ('poly'). Ignored by all other kernels
            gamma='auto', # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'; if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma; if 'auto', uses 1 / n_features
            coef0=0.0, # Independent term in kernel function. It is only significant in 'poly' and 'sigmoid'
            shrinking=True, 
            probability=False, # Whether to enable probability estimates. This must be enabled prior to calling fit, will slow down that method as it internally uses 5-fold cross-validation
            class_weight='balanced',
            random_state=0, 
            decision_function_shape='ovr').fit(x_train, y_train)

print(model.score(x_train, y_train))
model.support_vectors_

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'C': [100, 10, 1], 
     'kernel': ['linear', 'rbf', 'poly'], 
     'class_weight': [{0: 1.2, 1: 1.0, 2: 2.0}, 'balanced']},
]

# Can't use "auc_scorer" as we did above, as SVC can't give probability
def precision_2(y_true, y_pred):
    return precision_score(y_true, y_pred, average=None)[2]
def recall_2(y_true, y_pred):
    return recall_score(y_true, y_pred, average=None)[2]
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)
precision_scorer = make_scorer(precision_2, greater_is_better=True)
recall_scorer = make_scorer(recall_2, greater_is_better=True)
scoring = {'Accuracy': accuracy_scorer, 'Precision': precision_scorer, 'Recall': recall_scorer}

rs = RandomizedSearchCV(
    SVC(random_state=0),
    param_distributions=param_grid,
    n_iter=5,
    scoring=scoring,
    refit='Accuracy',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_Accuracy', 'std_test_Accuracy', 'mean_test_Precision', 'std_test_Precision', 'mean_test_Recall', 'std_test_Recall']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_Accuracy', 'mean_test_Precision', 'mean_test_Recall'], ascending=False)
df_cv

### Evaluation

In [ ]:
# proba_pred = model.predict_proba(x_test)
y_pred = model.predict(x_test)
acc_test = model.score(x_test, y_test)

# assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=3)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1', 'class 2']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=3)

## Linear model + regression 

### Load training and testing data

In [ ]:
folder = 'reg_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze() / 100000  # Dividing it by 100000 will significantly improve the following result
y_test = y_test.squeeze() / 100000  

In [ ]:
x_train

In [ ]:
y_train

### Train the model once

In [ ]:
# Train the model once
"""
a * ||w||_1 + 0.5 * b * ||w||_2^2
alpha = a + b and l1_ratio = a / (a + b)
"""
model = ElasticNet(alpha=1, l1_ratio=0.5, fit_intercept=True, 
                   random_state=0, tol=1e-4, max_iter=1000).fit(x_train, y_train)  

print(model.score(x_train, y_train))
model.coef_

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'alpha': list(np.arange(0.001, 1, 0.05)), 'l1_ratio': list(np.arange(0, 1, 0.01))},  # In my case, alpha should not be too small, otherwise the matrix is singular and the optimization algorithm doesn't converge
]

scoring = {'MSE': make_scorer(mean_squared_error, greater_is_better=False)}  # As you can see in df_cv, make_scorer will multiply -1 to MSE 

rs = RandomizedSearchCV(
    ElasticNet(random_state=0),
    param_distributions=param_grid,
    n_iter=2000,
    scoring=scoring,
    refit='MSE',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_MSE', 'std_test_MSE']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_MSE'], ascending=False)
df_cv

### Evaluation

In [ ]:
y_pred = model.predict(x_test)
print(mean_squared_error(y_test, y_pred))
df_predict = pd.concat((y_test, pd.Series(y_pred)), axis=1)
df_predict

## Random forest + regression 

### Load training and testing data

In [ ]:
folder = 'reg_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze() / 100000  # Dividing it by 100000 will significantly improve the following result
y_test = y_test.squeeze() / 100000  

In [ ]:
x_train

In [ ]:
y_train

### Train the model once

In [ ]:
# Train the model once
model = RandomForestRegressor(n_estimators=100, 
                               criterion='mse',
                               max_depth=3, # Avoid overfitting by controling the complexity of the tree
                               max_leaf_nodes=None, # Avoid overfitting by controling the complexity of the tree
                               ccp_alpha=0.0, # Avoid overfitting by pruning the tree
                               min_samples_split=2, # Avoid overfitting by avoid splitting too much
                               min_samples_leaf=1, # Avoid overfitting by avoid splitting too much
                               min_weight_fraction_leaf=0.0, # Avoid overfitting by avoid splitting too much
                               min_impurity_decrease=0.0, # Avoid overfitting by avoid splitting too much
                               max_features=1 / 3, # Make sure trees are independent
                               bootstrap=True, # Make sure trees are independent
                               max_samples=None, # Make sure trees are independent
                               oob_score=True,
                               n_jobs=-1,
                               random_state=0).fit(x_train, y_train)

print(model.score(x_train, y_train))
# When boostrap is True, samples may not be equal to the sum of values in each node, as "samples" is the "unique" number of samples, but "value" is the number of "repeated" samples
# https://stackoverflow.com/questions/56103507/why-does-this-decision-trees-values-at-each-step-not-sum-to-the-number-of-sampl
plt.rcParams["figure.figsize"] = (60, 60)
tree.plot_tree(model.estimators_[0], feature_names=x_train.columns)  # Visualize one of the trees in the random forest
plt.show()

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'n_estimators': [100, 200], 
     'criterion': ['mse'], # Using 'mae' here will make the following CV very slow, I don't know why
     'max_depth': [3, 6], 
     'min_samples_split': [2, 10],
     'min_samples_leaf': [1, 5],
     'min_impurity_decrease': [0, 0.02]}
]

scoring = {'MSE': make_scorer(mean_squared_error, greater_is_better=False)}  # As you can see in df_cv, make_scorer will multiply -1 to MSE 

rs = RandomizedSearchCV(
    RandomForestRegressor(random_state=0, max_features=1 / 3),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scoring,
    refit='MSE',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_MSE', 'std_test_MSE']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_MSE'], ascending=False)
df_cv

### Evaluation

In [ ]:
y_pred = model.predict(x_test)
print(mean_squared_error(y_test, y_pred))
df_predict = pd.concat((y_test, pd.Series(y_pred)), axis=1)
df_predict

## GBDT + regression 

### Load training and testing data

In [ ]:
folder = 'reg_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze() / 100000  # Dividing it by 100000 will significantly improve the following result
y_test = y_test.squeeze() / 100000  

In [ ]:
x_train

In [ ]:
y_train

### Train the model once

In [ ]:
# Train the model once
model = GradientBoostingRegressor(loss='ls', 
                                  learning_rate=0.1, # Learning rate shrinks the contribution of each tree by learning_rate. The larger the learning rate is, the fewer estimators we need
                                  n_estimators=100, # Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance
                                  criterion='friedman_mse',
                                  init='zero', # An estimator object that is used to compute the initial predictions
                                  max_depth=3, # Avoid overfitting by controling the complexity of the tree
                                  max_leaf_nodes=None, # Avoid overfitting by controling the complexity of the tree
                                  ccp_alpha=0.0, # Avoid overfitting by pruning the tree
                                  min_samples_split=2, # Avoid overfitting by avoid splitting too much
                                  min_samples_leaf=1, # Avoid overfitting by avoid splitting too much
                                  min_weight_fraction_leaf=0.0, # Avoid overfitting by avoid splitting too much
                                  min_impurity_decrease=0.0, # Avoid overfitting by avoid splitting too much
                                  subsample=1.0,  # The fraction of samples to be used for fitting the individual base learners. If smaller than 1.0 this results in Stochastic Gradient Boosting. subsample interacts with the parameter n_estimators. Choosing subsample < 1.0 leads to a reduction of variance and an increase in bias
                                  max_features=None, # Make sure trees are independent
                                  random_state=0).fit(x_train, y_train)

print(model.score(x_train, y_train))
tree.plot_tree(model.estimators_[0][0], feature_names=x_train.columns)  # Visualize one of the trees in the random forest, where now model.estimators_ is an array of size n_estimators * loss_.K
plt.show()

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'learning_rate': [0.01, 0.1],
     'n_estimators': [50, 100], 
     'max_depth': [3, 6], 
     'min_samples_split': [2, 10],
     'min_samples_leaf': [1, 5],
     'min_impurity_decrease': [0, 0.02], 
     'subsample': [1, 0.8]}
]

scoring = {'MSE': make_scorer(mean_squared_error, greater_is_better=False)}  # As you can see in df_cv, make_scorer will multiply -1 to MSE 

rs = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=0),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scoring,
    refit='MSE',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_MSE', 'std_test_MSE']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_MSE'], ascending=False)
df_cv

### Evaluation

In [ ]:
y_pred = model.predict(x_test)
print(mean_squared_error(y_test, y_pred))
df_predict = pd.concat((y_test, pd.Series(y_pred)), axis=1)
df_predict

## XGBoost + regression 

### Load training and testing data

In [ ]:
folder = 'reg_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze() / 100000  # Dividing it by 100000 will significantly improve the following result
y_test = y_test.squeeze() / 100000  

In [ ]:
x_train

In [ ]:
y_train

### Train the model once

In [ ]:
# Reference: 
# https://xgboost.readthedocs.io/en/latest/parameter.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=feature_importances#xgboost.XGBClassifier
# https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html

model = xgb.XGBRegressor(
        n_estimators=4,
        max_depth=3,
        learning_rate=0.9,
        objective='reg:squarederror',
        booster='gbtree',
        tree_method='exact',
        gamma=0.0,
        min_child_weight=1.0,
        max_delta_step=0.0,
        reg_alpha=0.0,
        reg_lambda=0.5,
        random_state=0,
        use_label_encoder=False,
        missing=np.nan,
        importance_type=None,
)

model.fit(x_train, y_train, eval_metric='rmse')

print(model.score(x_train, y_train))

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

### Train the model and search hyperparameters by CV

In [ ]:
param_grid = [
    {'n_estimators': [5, 10, 15, 20], 
     'learning_rate': [0.3, 0.5, 0.7, 0.9, 1.0],
     'max_depth': [3, 6, 9, 12], 
     'gamma': [0.0, 0.5, 1.0],
     'min_child_weight': [1.0, 2.0],
     'max_delta_step': [0.0, 0.5], 
     'lambda': [0.0, 0.5, 1.0], 
     'alpha': [0.0, 0.5, 1.0], 
    }
]

scoring = {'MSE': make_scorer(mean_squared_error, greater_is_better=False)}  # As you can see in df_cv, make_scorer will multiply -1 to MSE 

rs = RandomizedSearchCV(
    xgb.XGBRegressor(use_label_encoder=False, random_state=0),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scoring,
    refit='MSE',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train, eval_metric='rmse')

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_MSE', 'std_test_MSE']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_MSE'], ascending=False)
df_cv

### Evaluation

In [ ]:
y_pred = model.predict(x_test)
print(mean_squared_error(y_test, y_pred))
df_predict = pd.concat((y_test, pd.Series(y_pred)), axis=1)
df_predict

## MLP + regression 

### Load training and testing data

In [ ]:
folder = 'reg_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze() / 100000  # Dividing it by 100000 will significantly improve the following result
y_test = y_test.squeeze() / 100000  

In [ ]:
x_train

In [ ]:
y_train

### Train the model once

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

model = MLPRegressor(
    hidden_layer_sizes=(30, 20, 10),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size=256,
    learning_rate='constant', # Only used when solver='sgd'
    learning_rate_init=0.0001, # Only used when solver='sgd' or 'adam'
    power_t=0.5, # It is used in updating effective learning rate when the learning_rate is set to 'invscaling'. Only used when solver='sgd'
    max_iter=300, # For stochastic solvers ('sgd', 'adam'), note that this determines the number of epochs (how many times each data point will be used), not the number of gradient steps
    shuffle=True, # Whether to shuffle samples in each iteration. Only used when solver='sgd' or 'adam'
    random_state=0,
    early_stopping=False,
    validation_fraction=0.1 # The proportion of training data to set aside as validation set for early stopping. Must be between 0 and 1. Only used if early_stopping is True
).fit(x_train, y_train)

print(model.score(x_train, y_train))

print(model.n_layers_)
print(model.n_outputs_)
print(model.out_activation_)

In [ ]:
df_loss = pd.DataFrame({'n_epoch': list(range(1, len(model.loss_curve_) + 1)), 'train_loss': model.loss_curve_})
sns.lineplot(x='n_epoch', y='train_loss', data=df_loss)

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

param_grid = [
    {'hidden_layer_sizes': list(itertools.product(*[[40, 30], [30, 20], [20, 10, 5]])), 
     'alpha': [0.001, 0.0001], 
     'batch_size': [512, 256], 
     'learning_rate_init': [0.01, 0.001]},
]

scoring = {'MSE': make_scorer(mean_squared_error, greater_is_better=False)}  # As you can see in df_cv, make_scorer will multiply -1 to MSE 

rs = RandomizedSearchCV(
    MLPRegressor(max_iter=300, random_state=0),
    param_distributions=param_grid,
    n_iter=5,
    scoring=scoring,
    refit='MSE',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_MSE', 'std_test_MSE']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_MSE'], ascending=False)
df_cv

### Evaluation

In [ ]:
y_pred = model.predict(x_test)
print(mean_squared_error(y_test, y_pred))
df_predict = pd.concat((y_test, pd.Series(y_pred)), axis=1)
df_predict

## SVR + regression 

### Load training and testing data

In [ ]:
folder = 'reg_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze() / 100000  # Dividing it by 100000 will significantly improve the following result
y_test = y_test.squeeze() / 100000  

In [ ]:
x_train

In [ ]:
y_train

### Train the model once

In [ ]:
model = SVR(C=1.0, 
            kernel='rbf',
            degree=3, # Degree of the polynomial kernel function ('poly'). Ignored by all other kernels
            gamma='auto', # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'; if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma; if 'auto', uses 1 / n_features
            coef0=0.0, # Independent term in kernel function. It is only significant in 'poly' and 'sigmoid'
            epsilon=0.1, # Epsilon in the epsilon-SVR model. It specifies the epsilon-tube within which no penalty is associated in the training loss function with points predicted within a distance epsilon from the actual value
            shrinking=True).fit(x_train, y_train)

print(model.score(x_train, y_train))
model.support_vectors_

### Train the model and search hyperparameters by CV

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
# rank_test_score indicates the rank of a grid search parameter combination based on the mean_test_score

# This cell is too slow to run!
param_grid = [
    {'C': [100, 10, 1], 
     'kernel': ['linear', 'rbf', 'poly']},
]

scoring = {'MSE': make_scorer(mean_squared_error, greater_is_better=False)}  # As you can see in df_cv, make_scorer will multiply -1 to MSE 

rs = RandomizedSearchCV(
    SVR(),
    param_distributions=param_grid,
    n_iter=2,
    scoring=scoring,
    refit='MSE',
    return_train_score=False,
    cv=5, n_jobs=-1, random_state=0
)
rs.fit(x_train, y_train)

model = rs.best_estimator_
print(rs.best_estimator_)
print(rs.best_params_)

result = rs.cv_results_
list(result.keys())

In [ ]:
df_params = pd.DataFrame(result['params'])
df_metrics = pd.DataFrame({k: result[k] for k in ['mean_test_MSE', 'std_test_MSE']})
df_cv = pd.concat((df_params, df_metrics), axis=1).sort_values(by=['mean_test_MSE'], ascending=False)
df_cv

### Evaluation

In [ ]:
y_pred = model.predict(x_test)
print(mean_squared_error(y_test, y_pred))
df_predict = pd.concat((y_test, pd.Series(y_pred)), axis=1)
df_predict

# Recursive feature elimination (use random forest + binary classification as example)

## Load training and testing data

In [ ]:
folder = 'bin_clf_no_onehot'
x_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_train.csv'.format(folder), sep=',', index_col=False)
x_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/x_test.csv'.format(folder), sep=',', index_col=False)
y_train = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_train.csv'.format(folder), sep=',', index_col=False)
y_test = pd.read_csv('/Users/YouranQi/Documents/Career/ds_and_ml_interview/template/{:}/y_test.csv'.format(folder), sep=',', index_col=False)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [ ]:
x_train

In [ ]:
y_train.value_counts()

## Train the model once and get importance scores

In [ ]:
# Train the model once
model = RandomForestClassifier(n_estimators=100, 
                               criterion='gini',
                               max_depth=3, # Avoid overfitting by controling the complexity of the tree
                               max_leaf_nodes=None, # Avoid overfitting by controling the complexity of the tree
                               ccp_alpha=0.0, # Avoid overfitting by pruning the tree
                               min_samples_split=2, # Avoid overfitting by avoid splitting too much
                               min_samples_leaf=1, # Avoid overfitting by avoid splitting too much
                               min_weight_fraction_leaf=0.0, # Avoid overfitting by avoid splitting too much
                               min_impurity_decrease=0.0, # Avoid overfitting by avoid splitting too much
                               max_features='sqrt', # Make sure trees are independent
                               bootstrap=True, # Make sure trees are independent
                               max_samples=None, # Make sure trees are independent
                               oob_score=True,
                               class_weight='balanced',
                               n_jobs=-1,
                               random_state=0).fit(x_train, y_train)

print(model.score(x_train, y_train))
tree.plot_tree(model.estimators_[0], feature_names=x_train.columns)  # Visualize one of the trees in the random forest
plt.show()

In [ ]:
df_importance = pd.concat((pd.Series(x_train.columns), pd.Series(model.feature_importances_)), axis=1)
df_importance.columns = ['feature', 'importance_score']
df_importance = df_importance.sort_values(by='importance_score', ascending=False)
df_importance.plot(x='feature', y='importance_score', kind='bar', figsize=(20, 10))
df_importance

## Conduct RFE

In [ ]:
# For now, we didn't do hyperparameter search
# In practice, you should follow the steps mentioned at https://stats.stackexchange.com/a/323899

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, pos_label=1, average='binary')
recall_scorer = make_scorer(recall_score, pos_label=1, average='binary')

all_n_features_selected = list(reversed(range(1, df_importance.shape[0] + 1)))
all_scores = []
for n_features_selected in all_n_features_selected:
    features_selected = df_importance['feature'].iloc[:n_features_selected].tolist()
    scores = cross_val_score(model, 
                             x_train.loc[:, features_selected], 
                             y_train, 
                             scoring='roc_auc',  # can be 'roc_auc', accuracy_scorer, precision_scorer, recall_scorer
                             cv=5, n_jobs=-1, fit_params=None)
    all_scores.append(sum(scores) / len(scores))

In [ ]:
df_rfe = pd.DataFrame({'n_features': all_n_features_selected, 'scores': all_scores})
df_rfe.plot(x='n_features', y='scores', kind='line', figsize=(20, 10))
df_rfe

## Retrain with selected features

In [ ]:
features_selected = df_importance['feature'].iloc[:12].tolist()
x_train_selected = x_train.loc[:, features_selected]
x_test_selected = x_test.loc[:, features_selected]

model = model.fit(x_train_selected, y_train)

print(features_selected)
print(model.score(x_train_selected, y_train))
tree.plot_tree(model.estimators_[0], feature_names=x_train_selected.columns)  # Visualize one of the trees in the random forest
plt.show()

## Evaluation with selected features

In [ ]:
proba_pred = model.predict_proba(x_test_selected)
y_pred = model.predict(x_test_selected)
acc_test = model.score(x_test_selected, y_test)

assert (proba_pred.argmax(axis=1) == y_pred).all()
assert acc_test == accuracy_score(y_test, y_pred) == (y_test == pd.Series(y_pred)).mean()

In [ ]:
get_clf_metrics(y_test, y_pred, n_labels=2)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
get_confusion_matrix(y_test, y_pred, n_labels=2)

In [ ]:
plot_roc_curve(model, x_test_selected, y_test)

In [ ]:
plot_precision_recall_curve(model, x_test_selected, y_test)

# Conclusion

Talk about your final conclusions and business insights, depending on the goal of this data analysis.

You may say you tried several models, and their results are consistent: all models think several particular variables are important (according to which variables a linear model selects or the importance scores of a tree model).